# setup for colab

### enable GPU

You can enable the GPU in one of two ways: 

 * Select **`GPU`**  in the Edit->Notebook Settings for Hardware Accelerator   **OR**
 

 * Select **`GPU`**  in the Runtime->Change runtime type  for Hardware Accelerator 
 
 
 You should be able to now see the nvidia-smi diagnostic: 

In [ ]:
!nvidia-smi

Sat Nov 21 22:45:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

###  install bert-sklearn

In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 259 (delta 3), reused 3 (delta 0), pack-reused 247
Receiving objects: 100% (259/259), 519.36 KiB | 17.91 MiB/s, done.
Resolving deltas: 100% (125/125), done.
Processing /content/bert-sklearn
     |████████████████████████████████| 133kB 13.3MB/s 
     |████████████████████████████████| 6.9MB 25.6MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-cp36-none-any.whl size=54235 sha256=e165128709e82242ebd61062448d7c965d62c7fbcaf17d65893dc7206b73cc33
  Stored in directory: /root/.cache/pip/wheels/61/95/c6/5790aae8fb377f5ff356dbe58205aab28858595d6bff8197d0
Successfully built bert-sklearn
ERROR: botocore 1.19.23 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
['LICENSE', 'setup.

In [ ]:
import torch
print('pytorch version:', torch.__version__)
print('GPU:',torch.cuda.get_device_name(0))

pytorch version: 1.7.0+cu101
GPU: Tesla T4


In [ ]:
import os
import numpy as np
import pandas as pd
import re
from sklearn.metrics import classification_report
from tqdm import tqdm

from bert_sklearn import BertClassifier

In [ ]:
%%bash
if [[ ! -f ./zooniverse_all_data.csv ]]; then 
   wget https://raw.githubusercontent.com/aatishsuman/zooniverse-online-learning/master/data/zooniverse_all_data.csv
fi

if [[ ! -f ./zooniverse_train.csv ]]; then 
   wget https://raw.githubusercontent.com/aatishsuman/zooniverse-online-learning/master/data/zooniverse_train.csv
fi

if [[ ! -f ./humor_dataset.csv ]]; then 
   wget https://raw.githubusercontent.com/aatishsuman/zooniverse-online-learning/master/data/humor_dataset.csv
fi

--2020-11-21 22:45:28--  https://raw.githubusercontent.com/aatishsuman/zooniverse-online-learning/master/data/zooniverse_all_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17968115 (17M) [text/plain]
Saving to: ‘zooniverse_all_data.csv’

     0K .......... .......... .......... .......... ..........  0% 41.5M 0s
    50K .......... .......... .......... .......... ..........  0% 61.7M 0s
   100K .......... .......... .......... .......... ..........  0% 53.4M 0s
   150K .......... .......... .......... .......... ..........  1% 52.5M 0s
   200K .......... .......... .......... .......... ..........  1% 73.2M 0s
   250K .......... .......... .......... .......... ..........  1%  136M 0s
   300K .......... .......... .......... .......... ..........  1%  1

In [ ]:
all_data = pd.read_csv('zooniverse_all_data.csv')
test_df = pd.read_csv('zooniverse_train.csv')
train_df = pd.read_csv('humor_dataset.csv')

print(all_data.shape, test_df.shape, train_df.shape)
print(test_df.columns, train_df.columns)
train_df.head()

(90501, 8) (400, 16) (200000, 2)
Index(['web-scraper-order', 'text', 'section', 'topic', 'urls', 'userMentions',
       'isReply', 'emojis', 'Expression of emotions', 'Use of humor',
       'Self-disclosure', 'Referring explicitly to others' messages',
       'Asking questions',
       'Complimenting, expressing appreciation or criticism',
       'Expressing agreement or disagreement', 'Phatics, salutations'],
      dtype='object') Index(['text', 'humor'], dtype='object')


,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


In [ ]:
X_train, y_train = train_df['text'], train_df['humor'].astype(int)
print(len(X_train), len(y_train))
np.unique(y_train, return_counts=True)

200000 200000


(array([0, 1]), array([100000, 100000]))

In [ ]:
model = BertClassifier(max_seq_length = 128, epochs = 2, train_batch_size=32, validation_fraction=0.02)
print(model)
model.fit(X_train, y_train)

100%|██████████| 231508/231508 [00:00<00:00, 18157617.86B/s]

Building sklearn text classifier...
BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=None, epochs=2, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None, label_list=None, learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=32,
               use_cuda=True, validation_fraction=0.02, warmup_proportion=0.1)
Loading bert-base-uncased model...



100%|██████████| 433/433 [00:00<00:00, 91552.84B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 196000, validation data size: 4000



/content/bert-sklearn/bert_sklearn/model/pytorch_pretrained/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  next_m.mul_(beta1).add_(1 - beta1, grad)




Epoch 1, Train loss: 0.0619, Val loss: 0.0341, Val accy: 98.78%





Epoch 2, Train loss: 0.0150, Val loss: 0.0349, Val accy: 98.95%



BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=True, epochs=2, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None, label_list=array([0, 1]), learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=32,
               use_cuda=True, validation_fraction=0.02, warmup_proportion=0.1)

In [ ]:
import itertools
','.join(set(list(itertools.chain(*[emoji.split(';') for emoji in all_data['emojis'] if not pd.isnull(emoji)]))))

'😵,👌,😂,😲,👀,😀,😛,❤,👪,🙏,🎄,🐮,👯,😊,🐟,😦,😎,👍,🌌,📞,☺,▶,🙀,🙉,🎈,😭,🌞,❤️,🎆,👵,💇,🙋,😐,🐵,😒,🏃,✨,🎃,💆,💫,🎶,😜,😍,😪,😴,😌,🙊,😇,🐱,😺,↗️,😨,🍺,🙌,🎅,🍌,🔊,😕,👨,😄,🙎,😬,😑,😮,🎁,💻,🌊,😉,🚶,😅,☎,😸,☀️,😔,🎂,😁,😖,🌼,🎵,😃,👧,😣,🙈,👻,😏,😆'

In [ ]:
humor_emojis = ['😵','😂','😲','😛','🐮','😎','🙀','🙉','🐵','😒','🏃','🎃','😜','😍','😴','🙊','😇','🐱','😺','😄','😉','😅','😸','😁','🙈','👻','😏','😆']
def get_predictions(row):
  row['prediction'] = int(bool(row['prediction']) or any(i in humor_emojis for i in row['emojis'].split(';')) if not pd.isnull(row['emojis']) else False)
  return row

test_df['prediction'] = model.predict(test_df['text'].str.replace('\n',''))
test_df = test_df.apply(get_predictions, axis=1)
print(classification_report(test_df['Use of humor'], test_df['prediction'], target_names=['0', '1']))


              precision    recall  f1-score   support

           0       0.97      0.98      0.98       386
           1       0.31      0.29      0.30        14

    accuracy                           0.95       400
   macro avg       0.64      0.63      0.64       400
weighted avg       0.95      0.95      0.95       400



In [ ]:
test_df

,web-scraper-order,text,section,topic,urls,userMentions,isReply,emojis,Expression of emotions,Use of humor,Self-disclosure,Referring explicitly to others' messages,Asking questions,"Complimenting, expressing appreciation or criticism",Expressing agreement or disagreement,"Phatics, salutations",prediction
0,1604198662-928,Hi did you fix this bug?\n,bugreports,Subject 11057234,NaN,NaN,True,NaN,0,0,0,0,1,0,0,1,0
1,1604198796-1345,I have never been quite certain that the X ax...,bugreports,RFE: An easy way to read off feature frequencies,NaN,NaN,NaN,NaN,0,0,1,0,1,0,0,0,0
2,1604198776-1275,\nBecause I can understand it is of immense in...,bugreports,Statistics not updating on Gravity Spy home page,NaN,NaN,True,😦,1,0,1,0,0,0,0,0,1
3,1604198884-1593,"And working again.\n(A Schrödinger tool, as we...",bugreports,Gravity Spy Tools down,NaN,NaN,NaN,😃,1,1,1,0,0,0,0,0,1
4,1604197902-62,I did occasionally. Reloading the site helps t...,bugreports,Already Seen keeps coming up,/users/eperozzi;/users/sbc538,@eperozzi;@sbc538,NaN,NaN,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1604745498-12981,I think just a #1080 but a bit of #helix in th...,notes,Subject 5797761,/projects/zooniverse/gravity-spy/talk/subjects...,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0
396,1604529123-3128,#repeatingblips\n,notes,Subject 5864682,/projects/zooniverse/gravity-spy/talk/subjects...,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0
397,1604881637-6577,Another 1080 line with an amplitude envelope.\n,notes,Subject 5956891,/projects/zooniverse/gravity-spy/talk/subjects...,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0
398,1604538743-2305,Very low frequency blob 16-32Hz\n,notes,Subject 25771717,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0


In [ ]:
all_data['prediction'] = model.predict(all_data['text'].str.replace('\n',''))
all_data = all_data.apply(get_predictions, axis=1)

In [ ]:
predictions_df = all_data[all_data['prediction'] == 1]
print(len(predictions_df))
np.unique(predictions_df['section'], return_counts=True)

2006


(array(['bugreports', 'chat', 'collections', 'help', 'ligo', 'notes',
        'science', 'virgo'], dtype=object),
 array([  47,  118,   33,  108,   12, 1597,   89,    2]))

In [ ]:
all_data.to_csv('humor_predicted.csv', index=False)